In [1]:
import pdb
from math import log10, floor

import numpy as np
import scipy.optimize as op
import scipy.special as spec
from scipy import stats
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import seaborn as sns
import corner

import astropy
from astropy import stats as astrostats

import emcee
import radvel

%matplotlib inline

In [2]:
data = pd.read_csv('legacy_tables/system_props_719.csv')
specmatch = pd.read_csv('/Users/lee/Academics/Astronomy/Planets/legacy_analysis/legacy_tables/legacy_specmatch_medians.csv')
masses_only = pd.read_csv('legacy_tables/planet_list_719.csv')

In [3]:
masses_real = masses_only.query('status != "N" and status != "A"').reset_index(drop=True)
masses_good = masses_real.query('mass != "nan"').reset_index(drop=True)#[masses_only.hostname != 'hip63510']
masses_s = masses_good.query('status == "S" or status == "SS"').reset_index(drop=True)
masses_planet = masses_good.query('status != "S" and status != "SS"').reset_index(drop=True)
masses_old  = masses_planet.query('status == "K"').reset_index(drop=True)
masses_new  = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)

In [4]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_planet['hostname']
masses_table['Msini (MJup)'] = masses_planet['mass_med']
masses_table['a (AU)'] = masses_planet['axis_med']
masses_table['P (d)'] = masses_planet['per_med']
masses_table['e'] = masses_planet['e_med']

names = np.unique(np.array(masses_table.Name))

nplanets = np.array(masses_table.groupby('Name').count()['e'])
db_nplanets = pd.DataFrame({'Name':names, 'nplanets':nplanets})
masses_table = pd.merge(masses_table, db_nplanets, on='Name').reset_index()

In [5]:
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_planet.hostname))
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1

In [6]:
masses_planet['Name'] = masses_table['Name']

def round_to_1(x):
    return round(x, -int(floor(log10(x))))
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1) #log10(abs(x))
round_sig_vec = np.vectorize(round_sig)
def round_to_reference(x, y):
    return round(x, -int(floor(log10(y))))
round_to_ref_vec = np.vectorize(round_to_reference)

# Round errors and parameters with significant figures.
masses_planet['mass_err_plus_r']  = round_sig_vec(np.array(masses_planet['mass_plus'] - masses_planet['mass_med']))
masses_planet['mass_err_minus_r'] = round_sig_vec(np.array(masses_planet['mass_med'] - masses_planet['mass_minus']))
masses_planet['mass_err_av']      = round_sig_vec(0.5*(masses_planet['mass_err_plus_r'] + masses_planet['mass_err_minus_r']))
masses_planet['mass_med_r']       = round_to_ref_vec(np.array(masses_planet['mass_med']), masses_planet['mass_err_av'])

masses_planet['axis_err_plus_r']  = round_sig_vec(np.array(masses_planet['axis_plus'] - masses_planet['axis_med']))
masses_planet['axis_err_minus_r'] = round_sig_vec(np.array(masses_planet['axis_med'] - masses_planet['axis_minus']))
masses_planet['axis_err_av']      = round_sig_vec(0.5*(masses_planet['axis_err_plus_r'] + masses_planet['axis_err_minus_r']))
masses_planet['axis_med_r']       = round_to_ref_vec(np.array(masses_planet['axis_med']), masses_planet['axis_err_av'])

masses_planet['per_err_plus_r']   = round_sig_vec(np.array(masses_planet['per_plus'] - masses_planet['per_med']))
masses_planet['per_err_minus_r']  = round_sig_vec(np.array(masses_planet['per_med'] - masses_planet['per_minus']))
masses_planet['per_err_av']       = round_sig_vec(0.5*(masses_planet['per_err_plus_r'] + masses_planet['per_err_minus_r']))
masses_planet['per_med_r']        = round_to_ref_vec(np.array(masses_planet['per_med']), masses_planet['per_err_av'])

masses_planet['e_err_plus_r']     = round_sig_vec(np.array(masses_planet['e_plus'] - masses_planet['e_med']))
masses_planet['e_err_minus_r']    = round_sig_vec(np.array(masses_planet['e_med'] - masses_planet['e_minus']))
masses_planet['e_err_av']         = round_sig_vec(0.5*(masses_planet['e_err_plus_r'] + masses_planet['e_err_minus_r']))
masses_planet['e_med_r']          = round_to_ref_vec(np.array(masses_planet['e_med']), masses_planet['e_err_av'])
'''
masses_planet['tc_err_plus_r']     = round_sig_vec(np.array(masses_planet['tc_plus'] - masses_planet['tc_med']))
masses_planet['tc_err_minus_r']    = round_sig_vec(np.array(masses_planet['tc_med'] - masses_planet['tc_minus']))
masses_planet['tc_err_av']         = round_sig_vec(0.5*(masses_planet['tc_err_plus_r'] + masses_planet['tc_err_minus_r']))
masses_planet['tc_med_r']          = round_to_ref_vec(np.array(masses_planet['tc_med']), masses_planet['tc_err_av'])

masses_planet['w_err_plus_r']     = round_sig_vec(np.array(masses_planet['w_plus'] - masses_planet['w_med']))
masses_planet['w_err_minus_r']    = round_sig_vec(np.array(masses_planet['w_med'] - masses_planet['w_minus']))
masses_planet['w_err_av']         = round_sig_vec(0.5*(masses_planet['w_err_plus_r'] + masses_planet['w_err_minus_r']))
masses_planet['w_med_r']          = round_to_ref_vec(np.array(masses_planet['w_med']), masses_planet['w_err_av'])
'''
tex_writer = open('legacy_tables/planet_table.tex', 'w')
tex_writer.write(r'\begin{tabular}{lrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & \msini\ [\mjup] & a [AU] & P [d] & e \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in masses_planet.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['mass_med_r'], 
                                                   row['mass_err_plus_r'], 
                                                   row['mass_err_minus_r']) + r'$'
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['axis_med_r'], 
                                                   row['axis_err_plus_r'], 
                                                   row['axis_err_minus_r']) + r'$'
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['per_med_r'], 
                                                   row['per_err_plus_r'], 
                                                   row['per_err_minus_r']) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['e_med_r'], 
                                                   row['e_err_plus_r'], 
                                                   row['e_err_minus_r']) + r'$'
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{tabular}' + ' \n')
tex_writer.close()